In [4]:
#import library
import psycopg2
import pandas as pd
import csv
import math
import os
import re
import numpy as np

# LOAD DATA

In [84]:
#load data dari local ke python
path='E:/iykra/tugas/tugas python/home-credit-default-risk/'
#list file yang ada
listfile = os.listdir(path)
#load file 
for file in listfile:
    exec("%s = pd.read_csv('%s')" % (file.split(".")[0], path+file))

In [86]:
#mengambil sample 1000 row dari installment_payments
installments_payments = installments_payments.sample(n = 1000).reset_index(drop = True)

In [90]:
#memberikan nama kepada table
for file in listfile:
    exec("%s.name = '%s'" % (file.split(".")[0], file.split(".")[0]))#memberikan nama kepada table

# MEMBUAT CLASS JOIN

In [8]:
#membuat class join berisi metode untuk menggabungkan table
class join(object):
    def left(table1, table2, on_column):
        #memfilter table berdasarkan id
        id_values = table1[on_column].unique()
        result_temp = table2[table2[on_column].isin(id_values)]
        #left join table
        result = pd.merge(table1,
                     result_temp,
                     on = on_column,
                     how = 'left',
                     suffixes=('', '_'+table2.name))
        return result
            
    def full(table1,table2,on_column):
        #memfilter table berdasarkan id
        id_values = table1[on_column].unique()
        result_temp = table2[table2[on_column].isin(id_values)]
        #full join table
        result = pd.merge(table1,
                         result_temp,
                         on = on_column,
                         how = 'outer',
                         suffixes=('', '_'+table2.name))
        return result
        

# JOIN TABLE

In [9]:
#join table installment_payment dengan credit_card_balance
hasil1 = join.left(installments_payments, credit_card_balance, 'SK_ID_PREV')

In [10]:
#join table hasil 1 dengan POSH_CAHS_balance
hasil2 = join.left(hasil1, POS_CASH_balance, 'SK_ID_PREV')

In [11]:
#join table hasil 2 dengan previous_application
hasil3 = join.left(hasil2, previous_application, 'SK_ID_PREV')

In [12]:
#join table hasil 3 dengan application_train
hasil4 =  join.left(hasil3, application_train, 'SK_ID_CURR')

In [13]:
#join table hasil 4 dengan bureau
hasil5 = join.left(hasil4, bureau,'SK_ID_CURR')

In [14]:
#join table hasil 4 dengan bureau_balance
final_table = join.left(hasil5, bureau_balance,'SK_ID_BUREAU')

In [15]:
final_table

,index,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,SK_ID_CURR_credit_card_balance,...,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY_bureau,MONTHS_BALANCE_bureau_balance,STATUS
0,2954528,2060392,175047,1.0,5,-1391.0,-1401.0,9715.005,9715.005,NaN,...,0.0,21420.0,0.0,0.0,0.0,Consumer credit,-1519.0,NaN,NaN,NaN
1,2954528,2060392,175047,1.0,5,-1391.0,-1401.0,9715.005,9715.005,NaN,...,0.0,56790.0,NaN,NaN,0.0,Consumer credit,-954.0,NaN,NaN,NaN
2,2954528,2060392,175047,1.0,5,-1391.0,-1401.0,9715.005,9715.005,NaN,...,0.0,51241.5,0.0,0.0,0.0,Consumer credit,-74.0,NaN,NaN,NaN
3,2954528,2060392,175047,1.0,5,-1391.0,-1401.0,9715.005,9715.005,NaN,...,0.0,106871.4,0.0,0.0,0.0,Consumer credit,-608.0,NaN,NaN,NaN
4,2954528,2060392,175047,1.0,5,-1391.0,-1401.0,9715.005,9715.005,NaN,...,0.0,34920.0,0.0,0.0,0.0,Consumer credit,-501.0,NaN,NaN,NaN
5,2954528,2060392,175047,1.0,5,-1391.0,-1401.0,9715.005,9715.005,NaN,...,0.0,189000.0,51615.0,0.0,0.0,Consumer credit,-12.0,NaN,NaN,NaN
6,2954528,2060392,175047,1.0,5,-1391.0,-1401.0,9715.005,9715.005,NaN,...,0.0,38070.0,0.0,0.0,0.0,Consumer credit,-858.0,NaN,NaN,NaN
7,2954528,2060392,175047,1.0,5,-1391.0,-1401.0,9715.005,9715.005,NaN,...,0.0,21420.0,0.0,0.0,0.0,Consumer credit,-1519.0,NaN,NaN,NaN
8,2954528,2060392,175047,1.0,5,-1391.0,-1401.0,9715.005,9715.005,NaN,...,0.0,56790.0,NaN,NaN,0.0,Consumer credit,-954.0,NaN,NaN,NaN
9,2954528,2060392,175047,1.0,5,-1391.0,-1401.0,9715.005,9715.005,NaN,...,0.0,51241.5,0.0,0.0,0.0,Consumer credit,-74.0,NaN,NaN,NaN


# CREATE TABLE PADA POSTGRESQL

In [16]:
final_table.name = "application_train_final"

In [33]:
#create table dalam schema training
def create_table(table):
    conn = psycopg2.connect(host=host, database=database, user=user, password=password)
    cur = conn.cursor()
    dict = {'int64':'INTEGER','object':'VARCHAR','float64':'FLOAT',}
    col_type_temp = [ ]
    for i in range(len(table.keys())):
        a = table.keys()[i] +" "+ dict[str(table.dtypes[i])]
        col_type_temp.append(a)
    col_type = re.sub("\'|\[|\]", "", str(col_type_temp))
    command = "CREATE TABLE training."+table.name+"(%s)" % col_type
    cur.execute(command)
    conn.commit()
    cur.close()
    

In [46]:
#input database
host="localhost"
database="python"
user="postgres"
password="Pplatinum01"

In [28]:
#connect ke database
conn = psycopg2.connect(host=host, database=database, user=user, password=password)
cur = conn.cursor()

In [34]:
create_table(final_table)

# INSERT VALUES TO TABLE

In [61]:
#insert value, dicut sampai 1000 rows agar tidak Memory error
def insert_value(table):
    conn = psycopg2.connect(host=host, database=database, user=user, password=password)
    cur = conn.cursor()
    cols = ",".join([str(i) for i in table.columns.tolist()])
    for i,row in table[:1000].iterrows():
        sql2 = "INSERT INTO python.training."+ table.name +" (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
        cur.execute(sql2, tuple(row))
        conn.commit()
    cur.close()

In [36]:
insert_value(final_table)

# ALTER TABLE ADD PRIMARY KEY

In [88]:
#fungsi menambah Primary key
def add_pk(table, primary_key):
    conn = psycopg2.connect(host=host, database=database, user=user, password=password)
    cur = conn.cursor()
    command = 'ALTER table %s ADD PRIMARY KEY (%s)' %(table, primary_key)
    cur.execute(command)
    conn.commit()
    cur.close()


# LOAD DATA AWAL KE POSTGRES

In [93]:
listfile

['application_test.csv',
 'application_train.csv',
 'bureau.csv',
 'bureau_balance.csv',
 'credit_card_balance.csv',
 'installments_payments.csv',
 'POS_CASH_balance.csv',
 'previous_application.csv']

In [73]:
#create tables
for file in listfile:
    exec("create_table(%s)" %(file.split(".")[0]))

In [91]:
#insert values to table
insert_value(installments_payments)

In [94]:
insert_value(application_train)